<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2021notebooks/2021_1222BERT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- date: 2021_1222
- filename: 2021_1222BERT_demo.ipynb
- location: https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2021notebooks/2021_1222BERT_demo.ipynb
- author: 浅川伸一

# 新納(2001) の再現演習による BERT の理解

In [ ]:
import os
import sys
import numpy as np
from termcolor import colored

# 必要なライブラリのインストール
import platform
isColab = True if platform.system() == 'Linux' else False
if isColab:
    !pip install transformers > /dev/null 2>&1 

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1

In [ ]:
from transformers import BertJapaneseTokenizer
tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
tknz_ = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese', do_subword_tokenize=False)

In [ ]:
from termcolor import colored
# トークナイザの属性値を表示して理解を深める
for k in ['sep_token', 'sep_token_id', 'slow_tokenizer_class', 
          'unk_token', 'unk_token_id', 'verbose', #'vocab', 
          'vocab_files_names', 'vocab_size', 'word_tokenizer', 'word_tokenizer_type',
          'special_tokens_map', 'special_tokens_map_extended', 
          'subword_tokenizer', 'subword_tokenizer_type',
          'all_special_ids', 'all_special_tokens', 'unk_token', 'unk_token_id', 
          'special_tokens_map', 'special_tokens_map_extended', 
          'subword_tokenizer', 'subword_tokenizer_type', 
          'tokenize',
          'max_model_input_sizes', 'mecab_kwargs', 'model_input_names', 'model_max_length']:
    print(colored(f'{k}','yellow'), colored(f'{getattr(tknz,k)}','green'))

In [ ]:
#getattr(tknz, 'save_vocabulary')
#help(tknz.save_vocabulary)
tknz.save_vocabulary('vocab_saved.txt')  # 後に利用可能なように，語彙辞書をテキストファイルとして書き出す

# 結果の確認
!head vocab_saved.txt
!tail vocab_saved.txt

## 5.7 I BertForMaskedLM の利用 (新納2001, p.128)

BERT モデルはMasked Language Model によっても学習が行われているので、BERT のモデル自体に MASK された単語を推定する機構が含まれています。
MASK された単語を推定するには、モデルからその機構の部分を取り出さなければなりません。
これを行うのが ``BertForMaskedLM`` です。
例文「私は犬が好き。」の「犬」の部分を MASK した以下の単語列に対して、MASK の単語を推定してみます。

> 例文： 私は [MASK] が好き。

まずこの単語列をid 列に変換します(※5)

```python
ids = tknz.encode("私は[MASK]が好き。＂）
ids
[ 2, 1325, 9, 4, 14, 3596, 8, 3]
```

また、以下により [MASK] の位置を確認しておきます。

```python
mskpos = ids.index(tknz.mask_token_id)
mskpos
# 3
```

BERT モデルからの Masked Language Model の取り出しは以下のように行います。

```python
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')
```


次にモデルに単語のid 列を与えると、各単語の位置に現れる単語の分布が得られます。
```python
x = torch.LongTensor(ids).unsqueeze(O)
a = model(x)
```

上記に示したモデルからの出力 `a` は要素が 1つのタプルです。
`a[O]` の形状は以下のとおりです。

> ［ バッチサイズ， 単語列の長さ， 登録単語の数］

この例の場合、1 データだけなのでバッチサイズは 1、単語列の長さは 8 です。
そして登録単語の数はそのモデルの持つ登録単語数です。
この登録単語数は `tknz.vocab_size` から参照できます。
このモデルの場合は `32000` になっています。

```python
a[0].shape  # torch.Size([1, 8, 32000])
tknz.vocab_size # 32000
```

<font size="+1" color="green"> 新納本(2021) 近藤先生から自炊版をもらった pdf では，Bert の出力が 3 連 tuple と記載されている (p.124) が実際は違う
実際には 出力を .to_tuple() で tuple に変換しないといけない。</font>


この例の場合、MASK の位置は `mskpos` だったので、たとえば k=100 番目の登録単語が MASK の位置に現れる程度（確率）は以下のコードで得られます。
```python
k = 100
a[O][O][mskpos][k]
# tensor(-5.5299, grad_fn=<SelectBackward>)
```
変数 `k` を 0 から 31999 まで動かして最も大きな値を持つ $\hat{k}$ を求めれば、$\hat{k}$ 番目の登録単語が MASK の位置に最も高い確率で現れる単語と推定できます。
このようにベタに調べるよりも、torch には `topk` という便利なメソッドがあります。
これはベクトルの要素の中からその値の高いものを上から順に K 個取り出すものです。
以下のように使います。

```ptyhon
b = torch.topk(a[0][0[mskpos],k=5)
b[0]  # 上位 5 つの値
tensor([8.5864, 8.0724, 7.6974, 7.6480, 7.5863] ,...)
b[1］ ＃ 上位 5 つの index
tensor([1301, 1201, 705, 6968, 450])
```

In [ ]:
import torch
idxs = tknz.encode('私は[MASK]が好き。')
print(f'idxs :{idxs}')
maskpos = idxs.index(tknz.mask_token_id)
print(f'[MASK] の位置:{maskpos}')

from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')

x = torch.LongTensor(idxs).unsqueeze(0)
a = model(x)


print(a.to_tuple()[0].shape)  # torch.Size([1, 8, 32000])
print(tknz.vocab_size) # 32000

b = torch.topk(a.to_tuple()[0][0][maskpos],k=5)
print(b[0])     # 上位 5 つの値
# tensor([8.5864, 8.0724, 7.6974, 7.6480, 7.5863] ,...)

print(b[1])     # 上位 5 つの index
# tensor([1301, 1201, 705, 6968, 450])

for idx in b[1].detach().numpy():
    print('idx:', colored(f'{idx}', 'red' ,attrs=['bold']), 'トークン:', colored(f'{tknz.ids_to_tokens[idx]}','red', attrs=['bold']))

In [ ]:
# 近藤先生から送っていただいた，オノマトペ文章データを読み込む

#ファイルをアップロードします
from google.colab import files
files.upload()  # ご自身の PC からファイルをアップロードして下さい `original.csv`"

kondo_base = '.'
original = {}
n = 0
with open(os.path.join(kondo_base,'original.csv'), 'r', encoding='utf8') as f:
    s = f.read()
    for s_ in s.split('\n'):
        if n == 0:
            n += 1
            continue
        idx, sent = s_.split(',')
        original[int(idx)] = sent 

print(f'{len(original)} has been read')


In [ ]:
N = 5
for _ in range(N):
    N = np.random.randint(low=0, high=len(original))  # ランダムサンプリングしてみる
    sent = original[N]
    if '(' in sent:
        sent = sent.replace(')','').replace('(','')
    
    print(colored(sent, attrs=['bold']))    # 送っていただいた文
    print(colored('分かち書き','blue'), tknz.tokenize(sent)) # その分かち書き
    print(colored('分かち書き','green'), tknz_.tokenize(sent)) # その分かち書き
    print(colored('ID 化', 'blue'), tknz.encode(sent))   # 分かち書き結果の単語 ID 化
    print(colored('ID 化', 'green'), tknz_.encode(sent))   # 分かち書き結果の単語 ID 化


# MeCab で単語分割が行われて、MeCab が単語として認識しても、その単語が語鎮リスト vocab.txt に登録されていない場合は
# subword である WordPiece が起動され、その単語が適当に分割されます。そのように分割された単語には '##' が単語の前に付与されます。
# また、未知語の場合もWordPieceが起動され、同様に分割されます。

print(colored(f'\ntknz.all_special_ids:{tknz.all_special_ids}',attrs=['bold']))  #  [1, 3, 0, 2, 4]
print(colored(f'tknz.all_special_tokens:{tknz.all_special_tokens}', attrs=['bold']))  #  ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']